In [1]:
# this one is a total crab. read the next one.
# Understanding autoencoders via a another example---> simple solid boxes in an image 
# This time I want to apply variational autoencoder (VAE )

## generate the same dataset

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
import torch.nn as nn
import torch as T

/home/sobhan/miniconda3/envs/ml-dl/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
device = 'cpu'

In [4]:
from math import sqrt

def box(radius=2, box_size=28):
    '''
    to put some solid boxes in plain white images
    returns image(s) with boxes on them with the radius defined for them
    Comment: radius is a misnormer here. I wanted to show a scale thing for box in image
    firstly I wrote a cricle function (not a box) that is why I left radius like this 
    '''
    
    if isinstance(radius, int):
        rads = np.array([radius])
    elif not isinstance(radius, np.ndarray):
        rads = np.array(radius)
        
    rads = np.array(radius)

    position = np.zeros((rads.size, box_size//2 * 2, box_size//2 * 2))
    for i, r in enumerate(rads):
        
        for  x in range(-box_size//2, box_size//2):
            for y in range(-box_size//2, box_size//2):
                if ((abs(x) - r) < 0.01 and (abs(y) - r) < 0.01 ):
                    position[i, x+box_size//2 , y+box_size//2] = 1
   
    return  torch.from_numpy(position).flatten(start_dim=1)
    

In [5]:
box_siz = 10
num_features = 150   # sample data number
rand_radious= torch.randint(3, 50, (num_features,))/10 # create a random array of raduis
features = box(radius=rand_radious, box_size=box_siz)  # data set

In [6]:
# features[15]

In [7]:
labels = features # as an autoencoder we should have the same groud truth for output as input

In [8]:
features.shape, labels.shape

(torch.Size([150, 100]), torch.Size([150, 100]))

In [9]:
features[0], labels[0] # see? they are equal

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64))

## Reading the data set

In [10]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [11]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [12]:
data_iter

In [13]:
# it is import to use iterator for that
# next(iter(data_iter))

## Defining the Model

In [14]:
dim = (box_siz//2 * 2)**2  # square of image box. Used when the entire image is flattened or reshaped.

In [15]:
dim

100

In [16]:
# The bottleneck here has ... nodes

In [17]:
# inp = torch.tensor([1,2,3]).float()
# std = torch.exp(inp)

In [18]:
# std

In [19]:

# torch.randn_like(std)

In [23]:
input_size=dim

In [24]:
class VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(VariationalAutoencoder, self).__init__()
        self.fc1 = nn.Linear(input_size,512)
        self.fc21 = nn.Linear(512, 10)
        self.fc22 = nn.Linear(512, 10)
        
        
        self.relu = nn.ReLU()
        
        self.fc3 = nn.Linear(10, 512)
        self.fc4 = nn.Linear(512, input_size)
    
    def encode(self, x):
        x = self.relu(self.fc1(x))
        return self.fc21(x), self.fc22(x)
        
    def decode(self, z):
        z = self.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(z))
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 *logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
        
    def forward(self,x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        outp = self.decode(z)
#         print(outp)
        return outp, mu, logvar

In [25]:
# import torch.nn as nn
# import torch as T

# class Net(T.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.hid1 = T.nn.Linear(dim, 7)  
#         self.oupt = T.nn.Linear(7, dim)
    

#         T.nn.init.xavier_uniform_(self.hid1.weight)
#         T.nn.init.zeros_(self.hid1.bias)
#         T.nn.init.xavier_uniform_(self.oupt.weight)
#         T.nn.init.zeros_(self.oupt.bias)

#     def forward(self, x):
#         z = T.tanh(self.hid1(x))
#         z = self.oupt(z) 
        
#         return z

In [26]:
# net = Net()
net = VariationalAutoencoder()

In [27]:
# model:
net

VariationalAutoencoder(
  (fc1): Linear(in_features=100, out_features=512, bias=True)
  (fc21): Linear(in_features=512, out_features=10, bias=True)
  (fc22): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
  (fc3): Linear(in_features=10, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=100, bias=True)
)

## Initializing Model Parameters

In [28]:
# net[0] has accesss to the first layer of net

# net[0].weight.data.normal_(0, 0.01)
# net[0].bias.data.fill_(0)

## Defining the Loss Function

In [29]:
# loss = nn.MSELoss()

In [30]:
def vae_loss_fn(x, recon_x, mu, logvar):
    BCE = nn.functional. binary_cross_entropy(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return BCE + KLD

In [31]:
features[1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)

In [32]:
# testing loss function:
X = features[0].float()
y = X
net(X)
# loss(net(X), y)

(tensor([0.5109, 0.4580, 0.5003, 0.5627, 0.5680, 0.3785, 0.4814, 0.4127, 0.4702,
         0.4908, 0.4988, 0.4210, 0.5497, 0.5168, 0.4361, 0.5375, 0.4732, 0.5803,
         0.4796, 0.5303, 0.5207, 0.5648, 0.3986, 0.5807, 0.4446, 0.4149, 0.5404,
         0.5108, 0.4786, 0.4675, 0.4106, 0.4222, 0.4923, 0.4836, 0.5075, 0.4712,
         0.5036, 0.5171, 0.5652, 0.4504, 0.5013, 0.5096, 0.5027, 0.3844, 0.5814,
         0.5071, 0.4867, 0.4074, 0.4895, 0.5539, 0.4840, 0.5267, 0.4726, 0.4477,
         0.5318, 0.5791, 0.4887, 0.5226, 0.4454, 0.5814, 0.4350, 0.4545, 0.5661,
         0.6739, 0.4993, 0.4099, 0.4406, 0.5756, 0.5576, 0.5081, 0.4725, 0.4940,
         0.5450, 0.4761, 0.5053, 0.5411, 0.4537, 0.4501, 0.4711, 0.5592, 0.5179,
         0.4829, 0.4630, 0.5081, 0.4948, 0.4772, 0.4173, 0.5295, 0.4725, 0.4556,
         0.4984, 0.5530, 0.4739, 0.5341, 0.5382, 0.5118, 0.4494, 0.4586, 0.4909,
         0.5760], grad_fn=<SigmoidBackward0>),
 tensor([ 0.0471, -0.0731, -0.0219, -0.0912,  0.0269, -0.0256,

## Defining the Optimization Algorithm


In [33]:
# optimization algo
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [34]:
net.parameters()

<generator object Module.parameters at 0x7f7559b83cf0>

In [35]:
# list(net.parameters())

# evaluations

In [36]:
# losses = []

In [37]:
def calculate_loss(model, dataloader, loss_fn=nn.MSELoss(), flatten=True, conditional=False):
    losses = []
    for batch, labels in dataloader:
        batch = batch.to(device)
        labels = labels.to(device)
        
#         if flatten:
#             batch = batch.view(batch.size(0), 28*28)
            
#         if conditional:
#             loss = loss_fn(batch, model(batch, labels))
#         else:
        loss = loss_fn(batch, model(batch.float()))
            
        losses.append(loss)

    return (sum(losses)/len(losses)).item() # calculate mean



def evaluate(losses, autoencoder, dataloader, flatten=True, vae=False, conditional=False, title=""):
#     display.clear_output(wait=True)
    if vae and conditional:
        model = lambda x, y: autoencoder(x, y)[0]
    elif vae:
        model = lambda x: autoencoder(x)[0]
    else:
        model = autoencoder

    loss = calculate_loss(model, dataloader, flatten=flatten, conditional=conditional)
#     show_visual_progress(model, test_dataloader, flatten=flatten, vae=vae, conditional=conditional, title=title)
    
    losses.append(loss)
#     print(losses)



In [38]:
# features[0:5]

In [39]:
def evaluate(model):
        X_test = features[20:40] #.flatten()
        losses = []
        loss_fn=nn.MSELoss()
        for x in X_test:
            y_hat, mu, var = model(x.float())
#             loss_fn(model(x.float()), x.float())
            l = loss_fn(y_hat, x.float())
            losses.append(l)
        print(sum(losses))

# training

In [40]:
# test_dataloader = load_array((features[10:20], labels[10:20]), 5)

In [41]:
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [42]:
# loss_fn = nn.MSELoss()

In [43]:
num_epochs = 10
validation_losses = []
for epoch in range(num_epochs):
    for X,Y in data_iter:

        #l = loss(net(X.float()), Y.float())
        
        # +
        y_hat, mu, logvar = net(X.float())
#         print(X.float(), y_hat)
        l = vae_loss_fn(X.float(), y_hat, mu, logvar)

        trainer.zero_grad()
        l.backward()
        trainer.step() 
    

#     y_hat
#     evaluate(validation_losses, net, test_dataloader, vae=True, title=f'VAE - Epoch {epoch}')
    evaluate(net)
    
#         print(l.sum())
        
#         print(l.shape)
        #optimizer
#     l = (net(features.float()) - labels.float()).sum()
# #     print(net[0].weight.data)
#     if (epoch%10 == 0):
#         print(f'epoch {epoch + 1}, loss {l:f}')

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
# d = torch.tensor([1.15,1.1])
# loss_fn(d,d+1.1)

In [ ]:
box_siz//2 *2

# testing

In [ ]:
y.shape, mu.shape, var.shape

In [ ]:
# how our model represent a data sample like this: [0.1, 0.2, 0.3, 0.4]
sample_example = 2
x = features[sample_example].reshape(box_siz//2 *2,box_siz//2 *2).float()
y, mu, var = net(features[sample_example].float())
# y = mu + y*var
y = y.reshape(box_siz//2 *2,box_siz//2 *2).float()

In [ ]:
x # is also the ground truth as the same time (autoencoder)

In [ ]:
y

In [ ]:
x.shape, y.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.subplot(211)
plt.imshow(x.detach().numpy(), cmap='Greys')
plt.subplot(212)
plt.imshow(y.detach().numpy(), cmap='Greys',  interpolation='nearest')

In [ ]:

x = features[50].reshape(box_siz//2 *2,box_siz//2 *2).float()
y, mu, var = net(x.flatten())
y = y.reshape(box_siz//2 *2,box_siz//2 *2).float()

plt.subplot(211)
plt.imshow(x.detach().numpy(), cmap='Greys')
plt.subplot(212)
plt.imshow(y.detach().numpy(), cmap='Greys',  interpolation='nearest')

In [44]:
y.shape

torch.Size([100])

## adding noise to the data to see how it can recognize the original one

In [ ]:
x = torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0, 1., 0.],
        [0., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 1., 0., 0.,0., 1., 0.],
        [0., 0., 1.,0.,0.,0., 1., 0.],
        [0., 1., 1.,0., 0, 0., 1., 0.],
        [0., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

y, mu, var = net(x.flatten())
y = y.reshape(box_siz//2 *2,box_siz//2 *2).float()

plt.subplot(211)
plt.imshow(x.detach().numpy(), cmap='Greys')
plt.subplot(212)
plt.imshow(y.detach().numpy(), cmap='Greys',  interpolation='nearest')

In [ ]:
x = torch.tensor(  [[0., 0., 0., 0., 0., 0., 0., 0.],
                    [0., 0., 0., 0., 0., 0., 0., 0.],
                    [0., 0., 1., 1., 1., 1., 1., 0.],
                    [0., 0., 1.,  1., 1., 1., 1., 0.],
                    [0., 0., 1., 0., 0., 0., 1., 0.],
                    [0., 0,  1., 0., 0., 0., 1., 0.],
                    [0., 0., 1., 1., 1., 1., 1., 0.],
                    [0., 0., 0.,  1., 1., 1., 0., 0.]])
y, mu, var = net(x.flatten())
y = y.reshape(box_siz//2 *2,box_siz//2 *2).float()

plt.subplot(211)
plt.imshow(x.detach().numpy(), cmap='Greys')
plt.subplot(212)
plt.imshow(y.detach().numpy(), cmap='Greys',  interpolation='nearest')

# Notes

In [ ]:
# As a rule of thumb, remember this: The smaller the bottleneck, the lower the risk of overfitting. 
# but not too small.

## Advance

In [ ]:
# noww let's write a model with encoder and decoder like above but in an advance fashion understanding
# I don't too much in this just split the mentioned model into encoder and decoder functions:
# Also rename it from Net into Autoencoder class.

In [ ]:


class Autoencoder(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.hid1 = T.nn.Linear(dim, 7)  
        self.oupt = T.nn.Linear(7, dim)
    

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)
    
    # +
    def encoder(self, x):
        return T.tanh(self.hid1(x))
    
    # +
    def decoder(self, z):
        return self.oupt(z)
    
    # *
    def forward(self, x):
        z = encoder(x)
        z = decoder(z) 
        return z
        
        

In [ ]:
# VAE are good for finding the nonlinear relationships in data
# They can be used when features are high and data samples are low (by reduction of the features 
# that is typically used in dimensinal reductions)---> In biology samples is used repetiviely
# see: https://www.frontiersin.org/articles/10.3389/fgene.2019.01205/full